# Minodes Fingerprint to Zone Classification Challenge

## Introduction

At Minodes, we provide insights to retailers using WiFi Analytics. Insights are derived by analysing the location of people's smartphones during their visits.

How does it work? We install a set of WiFi routers (so called "nodes") with a custom firmware inside or close to the areas to be monitored. Nodes collect the signal strength (`RSSI`) of WiFi probe requests sent regularly by smartphones (so called "observations"). Observations are then used to estimate the position of smartphones: Let `RSSI(N,X)` be a function that returns the signal strength of probe request `X` from node `N` (measured in dBm: `0` is strongest signal strength, `-100` is weakest signal strength, extremes are seldomly reached). The set of observations `RSSI(Ni,X)` for all nodes `i` is called "fingerprint" and constitutes a radio signature of the smartphone correlated (non linearly) to its position in space (a distance of few meters translates to a signal strength within `[0,-30]`). Stores are manually partitioned into regions (so called "zones"), e.g., "entrance" and "checkout area". Fingerprints and zones are used as features and prediction classes, respectively.

Example: Given a store with zones `{A,B}`, nodes `{N1 (inside A), N2 (inside B), N3 (between A and B)}`, we want to estimate the most likely zone where the device that generated probe request `X` is located at that point of time. Intuitively, if `RSSI(N1,X) > RSSI(N3,X)`, the device is probably located in zone `A`. If `RSSI(N3,X)` is the highest, the answer is uncertain and we must look at the relative difference of the values of `RSSI(N1,X), RSSI(N3,X)`. If `RSSI(N2,X) > RSSI(N1,X)`, then most likely the correct zone is `B`.

## Dataset

This archive contains a dataset in CSV format located at `data/fingerprints_gt_ver3.csv`. The dataset represents a large deployment in a mall (three floors), and has been generated by walking inside each zone with a set of phones, labeling fingerprints manually with their corresponding zone. Nodes are located at the exterior of the entrance of the stores, and not inside the stores. Zones map to shops, aisles, and surrounding areas. The dataset is structured as follows:

| fr_observation_time  | fr_values  | fr_mac_address_id | zo_name  |
| -------------------- | ---------- | ----------------- | ---------|  
| 2015-12-08 10:00:13  | {'9': '-83', '13': '-67', '33': '-62', '101': ...  | 3192369 | Zone 355  |
| 2015-12-08 10:00:13  | {'12': '-69', '33': '-61', '128': '-68', '276'...  | 2002427 | Zone 355  |

Description of the fields:

* `fr_observation_time`: first timestamp in ascending order of the observations aggregated into the fingerprint (aggregation spans 1 second by default, to accomodate time desynchronisation issues on the nodes)
* `fr_values`: fingerprint, represented as a dictionary  {`node_id`: `signal_strength`, ...}. If a node ID is not present,  you can assume that its associated signal strength is `-100`
* `fr_mac_address_id`: unique id of the mac address of the phone that emitted the corresponding probe request
* `zo_name`: zone name (class to be predicted). Zone names are strings containing numbers, and do not have a direct semantic meaning.

Some statistics about the dataset:

* `343449` unique fingerprints (rows in the CSV file, excluding header line)
* `19` unique mac address ids. That means that 19 different phones where used to collect the dataset
* `449` unique zones
* `261` unique nodes
* On average, each node is present in `23224` fingerprints

## Problem

Given a fingerprint, predict its corresponding zone with a classifier.
The dataset can be used for training and testing the classifier.
Assess your solution with cross-validation by reporting results for confusion matrix, precision, recall and F1 score.

## Submission

Submit your solution by emailing us the link to a public GitHub repository, containing the solution as a Jupyter notebook and any additional files you deem necessary.
Do not include the files contained in this archive in the public repository.
You must use Python 3.x for your solution.
We evaluate your submission by scoring it on these questions:

* Is the candidate following the provided instructions?
* Is there an accompanying README.md file, that describes the contents of the submission?
* Is it well written using Markdown title(s), lists, and punctuation? any typos?
* Is it well structured, with a title, introduction, conclusions, sections, and explanatory comments?
* Is it easy to follow, with a clear and logical flow? Can I comprehend it without spending lots of energy?
* Last but not least: How good are results?

If you have any questions, feel free to approach us directly.

# Garret's Solution

## Approach
I chose five candidate classification algorithms. Four of the algorithms use decision tree estimators to predict labels. Decision trees were predominantely chosen as these estimators can natively handle multiclass outcomes with a high number of level (e.g. 449 unique zones in current data) without requiring one-versus-all, which would be computationally expensive for this number of levels. The fifth algorithm is a Support Vector Machines (SVM) that was chosen as it performs well across a diversity of problems. 

The models are as follows:

1. Extra Trees:  computationally cheap algorithm that randomly chooses cut points (lower variance), as opposed to Random Forests that estimated optimum tree cut points and is therefore computationally more expensive (lower bias).
Random Forests: a simple to use algorithm that requires little tuning.


2. Random Forests: ensemble method of 'bagging' of decision trees, which averages of numerous randomized (in terms of features selection used for estimated fit) tree fits, and this reduces the influence of overfitting trees for a more accurate and generalizable estimator.


3-4. Extreme Gradient Boosted Trees (including both Adaboost and XGB implementations of this algorithm): instead of averaging decision trees made in parallel using bagging as per Random Forests, this algorithm builds trees sequentially and therefore learns iteratively.


5. SVM: 

## Steps
Description of steps in numbered sections in analysis (steps 8-10 done twice for each model):


1. Imported data from csv to pandas dataframe, and removed all columns except 


2. Imputed values of non-represented nodes (-100).


3. Converted zone codes to numbers in a continuous sequence for prediction labels.


4. Subsampled to 20% of total data set (equally from each zone) to save time for evaluating models during parameter tuning.


5. Removed features that aren't likely to inform predictions of zone location (assuming new fingeprints will differ in the id and time from training data).


6. Performed matrix algebra to end up with a sparse matrix (zeros where -100s were) to allow memory compression for faster training.


7. Split data into train and test sets.


8. Tuned parameters using grid search.


9. Evaluated and compared models based on requested metrics.


10. Conclusions on model comparisons and future directions

## Comments
* Models were run on an AWS EC2 c8.xlarge instance. Even with this high memory capacity, model training times were very slow (~1.5 hours). For this reason, several model types (e.g. Random Forest, SVM) returned memory errors, even when trained on small fractions of the data. This was partially fixed by compressing the sparse data matrix. Training times were further speeded up by conducted grid search using only 5-fold CV and on only 20% of the data set, sampled equally from each zone.

### 1. Import data

In [1]:
# set paths
import os
import pandas as pd
import numpy as np
import csv
import ast

# import data
path = os.getcwd()
df_dir = os.path.join(os.path.sep.join(path.split(os.path.sep)),'data')
df = pd.read_csv(os.path.join(df_dir,'fingerprints_gt_ver3.csv'), index_col = False, quotechar = '"') 

# remove double quotes around dict values
conv = dict(fr_values=ast.literal_eval)
df = pd.read_csv(os.path.join(df_dir,'fingerprints_gt_ver3.csv'),  converters=conv)

### 2. Impute values

In [2]:
# function to unpack dictionary keys as pandas columns and impute missing values
def unpack(df, column, fillna=None):
    """
    This function goes through the dictionary values in the fr_values column
    and unpacks them into new columns with key value as column name.
    """
    ret = None
    if fillna is None:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems()))], axis=1)
        del ret[column]
    else:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems())).fillna(fillna)], axis=1)
        del ret[column]
    return ret

# run function
df = unpack(df, 'fr_values', fillna=-100)

### 3. Convert zone codes

In [3]:
# recode zone names as numeric
df['zo_name'] = df['zo_name'].str.replace('[^\d.]', '')

# factorise and record labels for decoding later
cat_columns = ['zo_name']
label_list = []

for column in cat_columns:
    """
    This function converts column levels into sequenital and factorized 
    categorical levels, keeping original values in dict for later decoding.
    """

    df[column] = df[column].astype('category')
    label_list.append(dict( enumerate(df[column].cat.categories) ))
    df[column] = df[column].astype('category').cat.codes

### 4. Subsample data

In [4]:
import random
fraction = .25
df = df.groupby('zo_name', group_keys=False).apply(lambda x: x.sample(frac=fraction))
y = df.zo_name

### 5. Remove irrelevant features

In [5]:
df.drop(['zo_name','fr_mac_address_id','fr_observation_time'], axis=1, inplace=True)
X = df.astype('int8').values
X = X+100

### 6. Sparse matrix compression

from scipy.sparse import csr_matrix
X = csr_matrix(X)

### 7. Split train-test data sets

In [6]:
from sklearn.model_selection import train_test_split

# split train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0,train_size=0.8)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [7]:
from sklearn.model_selection import cross_val_score
from sklearn import metrics  
from sklearn.grid_search import GridSearchCV

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


### 8. Hyperparameter tuning

In [8]:
# from sklearn.model_selection import GridSearchCV

# class EstimatorSelectionHelper:
#     """
#     This is a wrapper that takes in parameter ranges and selected 
#     models, performs grid search, and outputs model scores.
#     """
#     def __init__(self, models, params):
#         if not set(models.keys()).issubset(set(params.keys())):
#             missing_params = list(set(models.keys()) - set(params.keys()))
#             raise ValueError("Some estimators are missing parameters: %s" % missing_params)
#         self.models = models
#         self.params = params
#         self.keys = models.keys()
#         self.grid_searches = {}

#     def fit(self, X, y, cv=3, n_jobs=18, verbose=1, scoring=None, refit=False):
#         for key in self.keys:
#             print("Running GridSearchCV for %s." % key)
#             model = self.models[key]
#             params = self.params[key]
#             gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
#                               verbose=verbose, scoring=scoring, refit=refit,
#                               return_train_score=True)
#             gs.fit(X,y)
#             self.grid_searches[key] = gs    

#     def score_summary(self, sort_by='mean_score'):
#         def row(key, scores, params):
#             d = {
#                  'estimator': key,
#                  'min_score': min(scores),
#                  'max_score': max(scores),
#                  'mean_score': np.mean(scores),
#                  'std_score': np.std(scores),
#             }
#             return pd.Series({**params,**d})

#         rows = []
#         for k in self.grid_searches:
#             print(k)
#             params = self.grid_searches[k].cv_results_['params']
#             scores = []
#             for i in range(self.grid_searches[k].cv):
#                 key = "split{}_test_score".format(i)
#                 r = self.grid_searches[k].cv_results_[key]        
#                 scores.append(r.reshape(len(params),1))

#             all_scores = np.hstack(scores)
#             for p, s in zip(params,all_scores):
#                 rows.append((row(k, s, p)))

#         df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

#         columns = ['estimator', 'min_score', 'mean_score', 'max_score', 'std_score']
#         columns = columns + [c for c in df.columns if c not in columns]

#         return df[columns]

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier

# models1 = {
#     'ExtraTreesClassifier': ExtraTreesClassifier(),
#     'RandomForestClassifier': RandomForestClassifier(),
#     'AdaBoostClassifier': AdaBoostClassifier(),
#     'XGBClassifier': XGBClassifier(),
#     'SVC': SVC()
# }

# params1 = {
#     'ExtraTreesClassifier': { 'n_estimators': [16, 32] },
#     'RandomForestClassifier': { 'n_estimators': [16, 32] },
#     'AdaBoostClassifier':  { 'n_estimators': [16, 32] },
#     'XGBClassifier': { 'n_estimators': [16, 32], 'learning_rate': [0.8, 1.0] },
#     'SVC': [
#         {'kernel': ['linear'], 'C': [0.001, 0.01, 0.1]},
#     ]
# }

In [10]:
# helper1 = EstimatorSelectionHelper(models1, params1)
# helper1.fit(X_train, y_train, scoring='f1_macro', n_jobs=18)

In [11]:
#helper1.score_summary(sort_by='max_score')

### 9. Model evaluations


| estimator	| min_score	| mean_score | max_score | std_score | C |kernel | learning_rate | n_estimators |
|-----------|-----------|------------|-----------|-----------|---|-------|---------------|--------------|
| 3	| RandomForestClassifier	| 0.655098	| 0.657277	| 0.658536	| 0.00154728	| NaN	| NaN	| NaN	| 32 |
| 10  | SVC	| 0.622443	| 0.623999	| 0.626721	| 0.00193167	| 0.001	| linear	| NaN	| NaN|
| 2	|  RandomForestClassifier	| 0.618987	| 0.620894	| 0.62456	| 0.00259273	| NaN	| NaN	| NaN	| 16 |
| 1	|  ExtraTreesClassifier	| 0.611042	| 0.615962	| 0.623342	| 0.00531411	| NaN	| NaN	| NaN	| 32 |
| 11 | SVC	| 0.596674	| 0.597627	| 0.598637	| 0.000802158	| 0.01	| linear	| NaN	| NaN |
| 0	| ExtraTreesClassifier	| 0.570454	| 0.580172	| 0.589435	| 0.00775566	| NaN	| NaN	| NaN	| 16 |
| 12 | SVC	| 0.584213	| 0.585304	| 0.586462	| 0.000919383	| 0.1	| linear	| NaN	| NaN |
| 7	| XGBClassifier	| 0.0172699	| 0.035711	| 0.049727	| 0.0136149	| NaN	| NaN	| 0.8	| 32 |
| 9	| XGBClassifier	| 0.015628	| 0.0305242	| 0.0469324	| 0.0128246	| NaN	| NaN	| 1	| 32 |
| 6	| XGBClassifier	| 0.017612	| 0.0324528	| 0.0448098	| 0.0112415	| NaN	| NaN	| 0.8	| 16 |
| 8	| XGBClassifier	| 0.0112355	| 0.0223191	| 0.0374607	| 0.0110842	| NaN	| NaN	| 1	| 16 |
| 4	| AdaBoostClassifier	| 0.00295974	| 0.00417767	| 0.00584784	| 0.00122166	| NaN	| NaN	| NaN	| 16 |
| 5	| AdaBoostClassifier	| 0.00295974	| 0.00369243	| 0.00450662	| 0.000634136	| NaN	| NaN	| NaN	| 32 |


We can see from the table 

While Gradient Boosted Trees algorithms 

In [12]:
# print(__doc__)
# import itertools
# import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix

# def plot_confusion_matrix(cm, classes,
#                           normalize=False,
#                           title='Confusion matrix',
#                           cmap=plt.cm.Blues):
#     """
#     This function prints and plots the confusion matrix.
#     Normalization can be applied by setting `normalize=True`.
#     """
#     if normalize:
#         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
#     else:
#         print('Confusion matrix, without normalization')

#     print(cm)

#     plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()
#     tick_marks = np.arange(len(classes))
#     plt.xticks(tick_marks, classes, rotation=45)
#     plt.yticks(tick_marks, classes)

#     fmt = '.2f' if normalize else 'd'
#     thresh = cm.max() / 2.
#     for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#         plt.text(j, i, format(cm[i, j], fmt),
#                  horizontalalignment="center",
#                  color="white" if cm[i, j] > thresh else "black")

#     plt.tight_layout()
#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')

# # Compute confusion matrix
# cnf_matrix = confusion_matrix(y_test, y_pred)
# np.set_printoptions(precision=2)

# # Plot non-normalized confusion matrix
# plt.figure()
# plot_confusion_matrix(cnf_matrix, classes=class_names, title='Confusion matrix, without normalization')

# # Plot normalized confusion matrix
# plt.figure()
# plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True, title='Normalized confusion matrix')
# plt.show()

In [13]:
# from xgboost import XGBClassifier as xgb
# from sklearn.model_selection import StratifiedKFold

# learning_rate = [0.0001, 0.01, 1]
# param_grid = dict(learning_rate=learning_rate)
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
# clf_xgb = xgb(nthread=18, max_depth=2, silent=False, 
#               num_boost_round = 999, early_stopping_rounds=5,
#               objective='multi:softmax', num_class=449, 
#               eval_metric='mlogloss')
# grid_search = GridSearchCV(clf_xgb, param_grid, cv=kfold.split(X_train, y_train), n_jobs=-1)
# grid_result = grid_search.fit(X_train, y_train)

In [14]:
from sklearn.ensemble import RandomForestClassifier as rf

# clf_rf = rf(n_jobs=-1) #, max_features='sqrt', oob_score = True) 
 
# # Use a grid over parameters of interest
# param_grid = {"n_estimators" : [50,100,200]}
 
# cv_rfc = GridSearchCV(estimator=clf_rf, param_grid=param_grid, cv = 5)
# cv_rfc.fit(X_train, y_train)
# print(cv_rfc.best_params_)

In [15]:
# from xgboost import XGBClassifier as xgb
# clf_xgb = xgb(n_estimators=100,nthreads=18,max_depth=2,eta=0.1, silent=False, 
#               num_boost_round = 999, early_stopping_rounds=10,
#               objective='multi:softmax', num_class=449, 
#               eval_metric='mlogloss').fit(X_train,y_train)

# # make predictions for test data
# y_pred = clf_xgb.predict(X_test)
# print("Accuracy: %.2f%%" % (metrics.accuracy_score(y_test, y_pred) * 100.0))

In [16]:
# clf_rf = RandomForestClassifier(n_jobs=18,max_features='sqrt', oob_score = True) 
 
# # Use a grid over parameters of interest
# param_grid = {"n_estimators" : [128,256]}
 
# cv_rf = GridSearchCV(estimator=clf_rf, param_grid=param_grid, cv = 3)
# cv_rf.fit(X_train, y_train)
# print(cv_rf.best_params_)
#print(cv_rf.best_score)

In [ ]:
# fit xtra
clf_rf = RandomForestClassifier(n_estimators=256,n_jobs=18)
clf_rf.fit(X_train, y_train)
y_predict = clf_rf.predict(X_test)

print("accuracy: ", metrics.accuracy_score(y_test, y_predict))  
print("precision: ", metrics.precision_score(y_test, y_predict, average='macro'))  
print("recall: ", metrics.recall_score(y_test, y_predict, average='macro'))  
print("f1: ", metrics.f1_score(y_test, y_predict, average='macro'))  

In [ ]:
# # fit RF
# clf_xtra = RF(n_estimators=100)
# clf_xtra.fit(X_train, y_train)
# y_predict = clf_xtra.predict(X_test)

# print("accuracy: ", metrics.accuracy_score(y_test, y_predict))  
# print("precision: ", metrics.precision_score(y_test, y_predict, average='macro'))  
# print("recall: ", metrics.recall_score(y_test, y_predict, average='macro'))  
# print("f1: ", metrics.f1_score(y_test, y_predict, average='macro'))  

### 8. Hyperparameter tunining

In [ ]:
# from sklearn import svm
# from sklearn.model_selection import GridSearchCV
# def svc_param_selection(X, y, nfolds):
#     Cs = [0.001, 0.0001, 0.00001,]
#     param_grid = {'C': Cs}
#     grid_search = GridSearchCV(svm.LinearSVC(), param_grid, cv=nfolds)
#     grid_search.fit(X, y)
#     grid_search.best_params_
#     return grid_search.best_params_

# best_parameters = svc_param_selection(X_test, y_test, 10)
# print(best_parameters)

In [ ]:
# from sklearn.svm import LinearSVC as SVC
# #from sklearn.ensemble import RandomForestClassifier as RF
# #from sklearn.neighbors import KNeighborsClassifier as KNN
# from sklearn.model_selection import cross_val_score
# from sklearn import metrics  

# # fit SVC
# clf_svc = SVC(dual=False,C=.0001) # from scikit-learn docs: Prefer dual=False when n_samples > n_features
# clf_svc.fit(X_train, y_train)

# # fit RF
# clf_rf = RF(n_estimators=100)
# clf_rf.fit(X_train, y_train)

In [ ]:
# y_predict = clf_svc.predict(X_test)

# print("accuracy: ", metrics.accuracy_score(y_test, y_predict))  
# print("precision: ", metrics.precision_score(y_test, y_predict, average='macro'))  
# print("recall: ", metrics.recall_score(y_test, y_predict, average='macro'))  
# print("f1: ", metrics.f1_score(y_test, y_predict, average='macro'))  
# #print("area under curve (auc): ", metrics.roc_auc_score(y_test, y_predict, average='macro'))  